In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/ST/stargan

Mounted at /content/drive
/content/drive/MyDrive/ST/stargan


In [2]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import LambdaLR
import matplotlib.pyplot as plt
import os

seed = 2710
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
class MultiBranchClassifier(nn.Module):
    def __init__(self, num_channels=3, num_domains=4, num_classes=5, num_timesteps=128):
        super(MultiBranchClassifier, self).__init__()
        # Shared layers as before
        self.conv1 = nn.Conv1d(num_channels, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(64)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.25)
        self.fc_shared = nn.Linear(num_timesteps * 8, 100)

        # Prepare class-specific branches as a single module with conditionally applied outputs
        self.fc_class_branches = nn.Linear(100, 50 * num_classes)
        self.fc_final = nn.Linear(50, num_domains * num_classes)

    def forward(self, x, class_ids):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.relu(self.fc_shared(x))

        # Process all class-specific branches simultaneously
        class_branches = self.fc_class_branches(x).view(x.size(0), -1, 50)
        class_outputs = class_branches[torch.arange(class_branches.size(0)), class_ids]

        # Final class-specific output
        final_outputs = self.fc_final(class_outputs.view(x.size(0), 50))
        return final_outputs.view(x.size(0), -1)


def split_data(x, y, k, test_size=0.2, random_state=seed):
    x_train, x_test, y_train, y_test, k_train, k_test = train_test_split(x, y, k,
                                                                         test_size=test_size,
                                                                         random_state=random_state,
                                                                         stratify=k,
                                                                         shuffle=True)
    return x_train, x_test, y_train, y_test, k_train, k_test


def setup_training(x_train, y_train, k_train, x_test, y_test, k_test, batch_size=64):
    # Convert numpy arrays to torch tensors
    x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    k_train_tensor = torch.tensor(k_train, dtype=torch.long)
    x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    k_test_tensor = torch.tensor(k_test, dtype=torch.long)

    # Create datasets and loaders
    train_dataset = TensorDataset(x_train_tensor, y_train_tensor, k_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = TensorDataset(x_test_tensor, y_test_tensor, k_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, test_loader


def evaluate_model(model, test_loader, loss_fn, device='cpu'):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for x_batch, y_batch, k_batch in test_loader:
            x_batch, y_batch, k_batch = x_batch.to(device), y_batch.to(device), k_batch.to(device)
            outputs = model(x_batch, y_batch)
            loss = loss_fn(outputs, k_batch)
            total_loss += loss.item()

            # Assuming outputs are logits and k_batch are the true labels
            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == k_batch).sum().item()
            total_predictions += k_batch.size(0)

    total_loss /= len(test_loader)
    accuracy = correct_predictions / total_predictions

    return total_loss, accuracy


def train_model(model, train_loader, test_loader, loss_fn, optimizer, epochs=300):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Set up linear learning rate decay
    lambda_lr = lambda epoch: 1 - epoch / epochs
    scheduler = LambdaLR(optimizer, lr_lambda=lambda_lr)

    for epoch in range(epochs):
        model.train()
        for x_batch, y_batch, k_batch in train_loader:
            x_batch, y_batch, k_batch = x_batch.to(device), y_batch.to(device), k_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch, y_batch)
            loss = loss_fn(outputs, k_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

        # Update learning rate
        scheduler.step()

    test_loss, test_accuracy = evaluate_model(model, test_loader, loss_fn, device=device)

    return test_loss, test_accuracy

In [4]:
# Train domain classifier on Dp (RealWorld)

dataset = 'realworld'

if dataset == 'realworld':
    dataset_name = 'realworld_128_3ch_4cl'
    num_df_domains = 10
    num_dp_domains = 5
    num_classes = 4

elif dataset == 'cwru':
    dataset_name = 'cwru_256_3ch_5cl'
    num_df_domains = 4
    num_dp_domains = 4
    num_classes = 5

with open(f'data/{dataset_name}.pkl', 'rb') as f:
    x, y, k = pickle.load(f)

with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
    fs = pickle.load(f)

x = x[fs == 0]
y = y[fs == 0]
k = k[fs == 0]
x = x[k >= num_df_domains]
y = y[k >= num_df_domains]
k = k[k >= num_df_domains] - num_df_domains
print(x.shape, y.shape, k.shape)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

test_losses = []
test_accuracies = []

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    k_train, k_test = k[train_index], k[test_index]

    model = MultiBranchClassifier(num_domains=num_dp_domains, num_classes=num_classes, num_timesteps=x_train.shape[2])

    train_loader, test_loader = setup_training(x_train, y_train, k_train, x_test, y_test, k_test)

    loss_fn = nn.CrossEntropyLoss()

    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    test_loss, test_accuracy = train_model(model, train_loader, test_loader, loss_fn, optimizer, epochs=300)

    print(f'\nFold {i + 1}: Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}\n')

    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

print(f'\nMean test loss: {np.mean(test_losses):.4f} ± {np.std(test_losses):.4f}')
print(f'Mean test accuracy: {np.mean(test_accuracies):.4f} ± {np.std(test_accuracies):.4f}')

(3866, 3, 128) (3866,) (3866,)
Epoch 1/300, Loss: 1.6522547006607056
Epoch 2/300, Loss: 1.2458293437957764
Epoch 3/300, Loss: 0.8871831893920898
Epoch 4/300, Loss: 0.5761458277702332
Epoch 5/300, Loss: 0.6825745701789856
Epoch 6/300, Loss: 0.4881318211555481
Epoch 7/300, Loss: 0.4839795231819153
Epoch 8/300, Loss: 0.537638783454895
Epoch 9/300, Loss: 0.2677634060382843
Epoch 10/300, Loss: 0.07896178960800171
Epoch 11/300, Loss: 0.33506110310554504
Epoch 12/300, Loss: 0.12277072668075562
Epoch 13/300, Loss: 0.156798854470253
Epoch 14/300, Loss: 0.09619100391864777
Epoch 15/300, Loss: 0.09893094003200531
Epoch 16/300, Loss: 0.07884689420461655
Epoch 17/300, Loss: 0.12691448628902435
Epoch 18/300, Loss: 0.08544363081455231
Epoch 19/300, Loss: 0.04938160628080368
Epoch 20/300, Loss: 0.03991696238517761
Epoch 21/300, Loss: 0.061292462050914764
Epoch 22/300, Loss: 0.0626136064529419
Epoch 23/300, Loss: 0.005019540898501873
Epoch 24/300, Loss: 0.003738630097359419
Epoch 25/300, Loss: 0.015102

In [5]:
# Train domain classifier on Dp (CWRU)

dataset = 'cwru'

if dataset == 'realworld':
    dataset_name = 'realworld_128_3ch_4cl'
    num_df_domains = 10
    num_dp_domains = 5
    num_classes = 4

elif dataset == 'cwru':
    dataset_name = 'cwru_256_3ch_5cl'
    num_df_domains = 4
    num_dp_domains = 4
    num_classes = 5

with open(f'data/{dataset_name}.pkl', 'rb') as f:
    x, y, k = pickle.load(f)

with open(f'data/{dataset_name}_fs.pkl', 'rb') as f:
    fs = pickle.load(f)

x = x[fs == 0]
y = y[fs == 0]
k = k[fs == 0]
x = x[k >= num_df_domains]
y = y[k >= num_df_domains]
k = k[k >= num_df_domains] - num_df_domains
print(x.shape, y.shape, k.shape)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

test_losses = []
test_accuracies = []

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    k_train, k_test = k[train_index], k[test_index]

    model = MultiBranchClassifier(num_domains=num_dp_domains, num_classes=num_classes, num_timesteps=x_train.shape[2])

    train_loader, test_loader = setup_training(x_train, y_train, k_train, x_test, y_test, k_test)

    loss_fn = nn.CrossEntropyLoss()

    initial_lr = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)

    test_loss, test_accuracy = train_model(model, train_loader, test_loader, loss_fn, optimizer, epochs=300)

    print(f'\nFold {i + 1}: Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}\n')

    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

print(f'\nMean test loss: {np.mean(test_losses):.4f} ± {np.std(test_losses):.4f}')
print(f'Mean test accuracy: {np.mean(test_accuracies):.4f} ± {np.std(test_accuracies):.4f}')

(9426, 3, 256) (9426,) (9426,)
Epoch 1/300, Loss: 1.3555920124053955
Epoch 2/300, Loss: 1.1432180404663086
Epoch 3/300, Loss: 0.9216431379318237
Epoch 4/300, Loss: 0.7166136503219604
Epoch 5/300, Loss: 0.5770012736320496
Epoch 6/300, Loss: 0.5265796184539795
Epoch 7/300, Loss: 0.39507994055747986
Epoch 8/300, Loss: 0.36389821767807007
Epoch 9/300, Loss: 0.3359745144844055
Epoch 10/300, Loss: 0.43258902430534363
Epoch 11/300, Loss: 0.2470264583826065
Epoch 12/300, Loss: 0.17564035952091217
Epoch 13/300, Loss: 0.16051918268203735
Epoch 14/300, Loss: 0.08320073783397675
Epoch 15/300, Loss: 0.14022196829319
Epoch 16/300, Loss: 0.11472320556640625
Epoch 17/300, Loss: 0.14710599184036255
Epoch 18/300, Loss: 0.11179430037736893
Epoch 19/300, Loss: 0.07077523320913315
Epoch 20/300, Loss: 0.08296822011470795
Epoch 21/300, Loss: 0.04832712188363075
Epoch 22/300, Loss: 0.036364439874887466
Epoch 23/300, Loss: 0.061778269708156586
Epoch 24/300, Loss: 0.044748563319444656
Epoch 25/300, Loss: 0.1097